<a href="https://colab.research.google.com/github/jivitheshreddy/INFO-5731-Srping2023/blob/main/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Question 1

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon.

(2) Collect the top 10000 User Reviews of a film recently in 2023 or 2022 (you can choose any film) from IMDB.

(3) Collect all the reviews of the top 1000 most popular software from [G2](https://www.g2.com/) or [Capterra](https://www.capterra.com/)

(4) Collect the abstracts of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from [Semantic Scholar](https://www.semanticscholar.org).

(5) Collect all the information of the 904 narrators in the [Densho Digital Repository](https://ddr.densho.org/narrators/).

(6) Collect the top 10000 tweets by using a hashtag (you can use any hashtag) from Twitter. 

In [ ]:
# Write your code here

import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium import webdriver
url="https://www.imdb.com/title/tt10640346/reviews?ref_=tt_sa_3"
import pandas as pd
import requests

In [ ]:
browser=webdriver.Firefox()
browser.get(url)
html=browser.page_source

In [ ]:
soup=BeautifulSoup(html,'html.parser')

In [ ]:
base_url="https://www.imdb.com/title/tt10640346/reviews?ref_=tt_sa_3"

In [ ]:
master = {}
from tqdm import tqdm

# Define a function to scrape all reviews from a page, including "load more" option
def scrape_reviews(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    review = []
    count = 0
    while True:
        resp = requests.get(url, headers=headers).text
        soup = BeautifulSoup(resp, 'html.parser')
        for i in soup.find_all("div", class_="content"):
            review.append(i.text.strip())
            count += 1
            if count == 10000:
                break
        if count == 10000:
            break
        next_link = soup.find("a", class_="page next")
        if next_link:
            url = next_link['href']
        else:
            break
    table = pd.DataFrame({'Review': review})
    return table

# Loop through all pages and scrape reviews
for i in tqdm(range(1, 500)):
    url = base_url.format(i)
    master[i] = scrape_reviews(url)


100%|████████████████████████████████████████████████████████████████████████████████| 499/499 [10:41<00:00,  1.29s/it]


In [ ]:
big_table=pd.concat([master[i] for i in master],axis=0)

In [ ]:
big_table=big_table.head(10000)

 **Question 2**


(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.


In [ ]:
#Removing Numbers and Puncuation
big_table['Review'] = big_table['Review'].str.replace('[^\w\s]','')
big_table['Review'].head()

C:\Users\jivit\AppData\Local\Temp\ipykernel_7904\14359710.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  big_table['Review'] = big_table['Review'].str.replace('[^\w\s]','')


0    Whether it be orgies showcasing various bodily...
1    This film felt like it was written and directe...
2    So many reviews praise this film for the level...
3    After an interesting opening scene about our m...
4    In the opening scene of BABYLON an elephant on...
Name: Review, dtype: object

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
big_table['Review'] = big_table['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
big_table['Review'].head()

0    Whether orgies showcasing various bodily fluid...
1    This film felt like written directed high scho...
2    So many reviews praise film level debauchery s...
3    After interesting opening scene main character...
4    In opening scene BABYLON elephant pickup empti...
Name: Review, dtype: object

In [ ]:
big_table['Review'] = big_table['Review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
big_table['Review'].head()

0    whether orgies showcasing various bodily fluid...
1    this film felt like written directed high scho...
2    so many reviews praise film level debauchery s...
3    after interesting opening scene main character...
4    in opening scene babylon elephant pickup empti...
Name: Review, dtype: object

Question 3
(30 points). Write a python program to conduct syntax and structure analysis of the clean text you just saved above. The syntax and structure analysis includes:

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
import pandas as pd
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Define the stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Define a function to apply the stemmer and lemmatizer to each word in a string
def stem_and_lemmatize(text):
    # Tokenize the text into words
    words = nltk.word_tokenize(text)
    # Stem and lemmatize each word
    stemmed_words = [stemmer.stem(word) for word in words]
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    # Join the stemmed and lemmatized words back into a string
    stemmed_text = ' '.join(stemmed_words)
    lemmatized_text = ' '.join(lemmatized_words)
    # Return the stemmed and lemmatized texts as a tuple
    return (stemmed_text, lemmatized_text)

# Apply the function to each row in the DataFrame
big_table[['stemmed_text', 'lemmatized_text']] = big_table['Review'].apply(stem_and_lemmatize).apply(pd.Series)

# Print the result
print(big_table)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jivit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\jivit\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


                                               Review  \
0   whether orgies showcasing various bodily fluid...   
1   this film felt like written directed high scho...   
2   so many reviews praise film level debauchery s...   
3   after interesting opening scene main character...   
4   in opening scene babylon elephant pickup empti...   
..                                                ...   
20  score 2010my brain hurts wow awful i wouldnt e...   
21  the good great character brad pitt adding dept...   
22  damien chazelle chosen tag along filmmakers de...   
23  i really want understand producers thought who...   
24  damian chazelle doesnt make bad movies whiplas...   

                                         stemmed_text  \
0   whether orgi showcas variou bodili fluid plot ...   
1   thi film felt like written direct high school ...   
2   so mani review prais film level debaucheri sho...   
3   after interest open scene main charact manni d...   
4   in open scene babylon elep

In [ ]:
big_table['lemmatized_text']

0     whether orgy showcasing various bodily fluid p...
1     this film felt like written directed high scho...
2     so many review praise film level debauchery sh...
3     after interesting opening scene main character...
4     in opening scene babylon elephant pickup empty...
                            ...                        
20    score 2010my brain hurt wow awful i wouldnt ev...
21    the good great character brad pitt adding dept...
22    damien chazelle chosen tag along filmmaker dec...
23    i really want understand producer thought who ...
24    damian chazelle doesnt make bad movie whiplash...
Name: lemmatized_text, Length: 10000, dtype: object

In [ ]:
text_data1 = ''
for index, value in big_table['lemmatized_text'].head(1000).iteritems():
    text_data1 += value + '\n'

In [ ]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.2.0/en_core_web_sm-3.2.0-py3-none-any.whl

     ---------------------------------------- 13.9/13.9 MB 9.5 MB/s eta 0:00:00


In [ ]:
import spacy

# Load the language model
nlp = spacy.load('en_core_web_sm')
nlp.max_length = 2000000

# Tokenize the text data
doc = nlp(text_data)

# Parts of Speech (POS) Tagging
noun_count = 0
verb_count = 0
adj_count = 0
adv_count = 0
for token in doc:
    if token.pos_ == 'NOUN':
        noun_count += 1
    elif token.pos_ == 'VERB':
        verb_count += 1
    elif token.pos_ == 'ADJ':
        adj_count += 1
    elif token.pos_ == 'ADV':
        adv_count += 1

print(f"Total number of Nouns: {noun_count}")
print(f"Total number of Verbs: {verb_count}")
print(f"Total number of Adjectives: {adj_count}")
print(f"Total number of Adverbs: {adv_count}")

# Named Entity Recognition
entities = {}
for ent in doc.ents:
    if ent.label_ in entities:
        entities[ent.label_] += 1
    else:
        entities[ent.label_] = 1

print("Named Entities:")
for label, count in entities.items():
    print(f"{label}: {count}")


Total number of Nouns: 74960
Total number of Verbs: 39880
Total number of Adjectives: 31520
Total number of Adverbs: 15400
Named Entities:
PERSON: 2280
GPE: 1680
TIME: 960
DATE: 1080
CARDINAL: 2680
ORG: 400
NORP: 240
ORDINAL: 560
FAC: 40


In [ ]:
#sing only first 10000 words as the systems is running out of memory
first_10000_words = text_data1[:10000]

In [ ]:
import spacy

# load the English language model
nlp = spacy.load('en_core_web_sm')


# perform constituency parsing
doc = nlp(first_10000_words)
print('Constituency parsing:')
for sent in doc.sents:
    for np in sent.noun_chunks:
        print(np.text, '=>', np.root.text)

# perform dependency parsing
doc = nlp(first_10000_words)
print('\nDependency parsing:')
for token in doc:
    print(token.text, '=>', token.dep_, '=>', token.head.text, '=>', [child.text for child in token.children])


Constituency parsing:
orgy => orgy
various bodily fluid plot thread runtime film damien chazelle => chazelle
fully unrestrained latest film => film
i => i
directorial style => style
he => he
flash brilliance => brilliance
babylon => babylon
extreme tendency => tendency
modern hollywood => hollywood
i => i
the opening sequence sight => sight
mesmerized vibrant energy => energy
the film => film
good pace => pace
two hour point => point
i => i
runtime => runtime
it => it
chazelle => chazelle
i => i
film => film
another scene => scene
the runtime => runtime
whole plot line => line
filmjustin hurwitz => hurwitz
another terrific score => score
especially bizarre movie montage end => end
really gel technical aspect film => film
quite achievementchazelle => achievementchazelle
someone => someone
some better editing => editing
combined selfrestraint => selfrestraint
much closer epic masterpiece status => status
he => he
a stand pretty entertaining tale excess fame => fame
early year hollywood =


Dependency parsing:
whether => mark => showcasing => []
orgy => nsubj => showcasing => []
showcasing => advcl => chazelles => ['whether', 'orgy', 'chazelle', 'film', 'whiplash']
various => amod => chazelle => []
bodily => advmod => fluid => []
fluid => amod => runtime => ['bodily']
plot => compound => runtime => []
thread => compound => runtime => []
runtime => compound => film => ['fluid', 'plot', 'thread']
film => compound => chazelle => ['runtime']
damien => compound => chazelle => []
chazelle => dobj => showcasing => ['various', 'film', 'damien']
fully => advmod => unrestrained => []
unrestrained => amod => film => ['fully']
latest => amod => film => []
film => dobj => showcasing => ['unrestrained', 'latest']
la => compound => land => []
la => compound => land => []
land => compound => whiplash => ['la', 'la']
whiplash => conj => showcasing => ['land', 'film']
favorite => amod => film => []
film => npadvmod => whiplash => ['favorite', 'm']
i => nsubj => m => []
m => relcl => film 

the => det => show => []
costuming => amod => show => []
set => amod => show => []
dynamic => amod => show => []
show => dobj => pull => ['the', 'costuming', 'set', 'dynamic']
disorganized => ccomp => thought => ['hollywood']
tumultuous => amod => hollywood => []
early => amod => hollywood => []
hollywood => dobj => disorganized => ['tumultuous', 'early']
i => nsubj => hooked => []
hooked => amod => conrad => ['i', 'presentation', 'keep', 'help', 'entertainedwhile', 'come', 'thing']
getgo => compound => movie => []
movie => compound => flaw => ['getgo']
flaw => compound => presentation => ['movie']
presentation => dobj => hooked => ['flaw']
vibrant => amod => fastmoving => []
fastmoving => nsubj => keep => ['vibrant']
keep => ccomp => hooked => ['fastmoving']
help => dobj => hooked => []
entertainedwhile => advcl => hooked => ['pacing']
babylon => nmod => pacing => []
distinctive => amod => pacing => []
look => compound => pacing => []
pacing => dobj => entertainedwhile => ['babylon', 

357 => nsubj => found => ['251']
found => relcl => check => ['357', 'review']
helpful => amod => review => []
wa => nmod => review => []
review => ccomp => found => ['helpful', 'wa']
helpful => amod => vote => []
sign => compound => vote => []
vote => nsubj => permalink => ['helpful', 'sign']
permalink => ccomp => hope => ['vote', '\n', 'in']

 => dep => permalink => []
in => prep => permalink => ['pickup']
opening => amod => pickup => []
scene => compound => babylon => []
babylon => compound => pickup => ['scene']
elephant => compound => pickup => []
pickup => pobj => in => ['opening', 'babylon', 'elephant']
empty => amod => bowel => []
bowel => advcl => hope => ['empty', 'handler', 'm']
one => nummod => handler => []
handler => dobj => bowel => ['one', 'this']
this => appos => handler => []
i => nsubj => m => []
m => conj => bowel => ['i', 'sorry']
sorry => acomp => m => []
say => compound => hollywoodland => ['tone', 'follows']
set => amod => tone => []
tone => dobj => say => ['set'

i => nsubj => left => []
left => ROOT => left => ['i', 'evening', 'watching', 'make', 'third']
theater => compound => evening => []
evening => dobj => left => ['theater']
watching => advcl => left => ['film', 'long']
film => dobj => watching => []
far => advmod => long => []
long => advmod => watching => ['far']
probably => advmod => one => []
one => nummod => storyline => ['probably']
storyline => nsubj => make => ['one', 'many']
many => amod => storyline => []
make => advcl => left => ['storyline', 'fly', 'film', 'within', 'felt']
movie => nsubj => fly => []
fly => ccomp => make => ['movie', 'instead', 'thing']
instead => advmod => fly => []
first => amod => thing => []
act => nmod => thing => []
drag => compound => drag => []
drag => nmod => dragsthe => ['drag']
dragsthe => nmod => thing => ['drag']
funny => amod => thing => []
thing => npadvmod => fly => ['first', 'act', 'dragsthe', 'funny']
best => amod => film => []
moment => compound => film => []
film => dobj => make => ['best'

**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

Constituency parsing and dependency parsing are two types of syntactic analysis used to understand the structure of a sentence.

Constituency parsing involves breaking down a sentence into its constituent parts, which are subphrases or constituents that form the sentence. These constituents are then arranged in a hierarchical structure called a parse tree. Each node in the parse tree represents a constituent, and its children represent its immediate subparts. For example, in the sentence "The cat sat on the mat", the noun phrase "the cat" and the prepositional phrase "on the mat" are constituents that can be broken down into smaller constituents. The parse tree shows the hierarchical relationship between these constituents and their subparts.

Dependency parsing, on the other hand, focuses on the grammatical relationships between words in a sentence. It represents a sentence as a directed graph where each word is a node, and the edges between nodes indicate the grammatical relationships between the words. For example, in the sentence "The cat sat on the mat", the word "cat" is the subject of the verb "sat", and the preposition "on" attaches the prepositional phrase "the mat" to the verb "sat". These relationships can be represented using labeled arcs in the dependency parse tree.

In summary, constituency parsing breaks down a sentence into its constituent parts and represents them in a hierarchical structure, while dependency parsing focuses on the grammatical relationships between words in a sentence and represents them as a directed graph. Both types of parsing can be used to analyze the syntactic structure of a sentence and can be useful in natural language processing tasks such as sentiment analysis and machine translation.